# Overview

The purpose of this example is to give our users a better understanding of how the roadnetwork is represented in BeamNG.
To do so we screate a scenario using the West Coast USA level, extract all geographic road data, and plot it.

## Setup

This example relies on different classes from the beamngpy module, the external library shapely, and matplotlib. 



In [1]:
from shapely.geometry import MultiLineString, LineString

from beamngpy import BeamNGpy, Vehicle, Scenario, Road

## Scenario Creation

A simple scenario is created in order to access the geographic data from the game.


In [3]:
beamng = BeamNGpy('localhost', 64256)

scenario = Scenario('west_coast_usa', 'road_map_example')
orig = (568.908386, 13.4217358,  148.56546)

vehicle = Vehicle('ego_vehicle', model='pickup', licence='PYTHON')
scenario.add_vehicle(vehicle, pos=orig)

scenario.make(beamng)

## Running the Scenario

With the scenario created, we finally start the simulator and load our created scenario. The scenario simulation does not start immediately after loading, but only when the user triggers the start using start_scenario().

In [4]:
bng = beamng.open()
bng.load_scenario(scenario)
bng.start_scenario()

## Extracting the Road's Data

At first we get the road's data. As we are only interested in their id's we discard any other information.
Every ID refers to one road. One road defined by a list of N nodes.
One such node is a triplet of xyz coordinates: ((x1, y1, z1), (x3, y3, z3), (x3, y3, z3)). This list of coordinates defines the leftmost, middle and rightmost point of such a node.

Because the goal is to plot a simple map of the level, we only use the middle coordinate of every node and disregard the y coordinate.

In [6]:
roads = bng.get_roads()
road_names = list(roads.keys())
road_spec = {}
for rid in road_names:
    road_spec[rid] = bng.get_road_edges(rid)

road = list()
lines = list()
for r_id in road_spec.keys(): # this may take a while
    road = list()
    for r_point in road_spec[r_id]:
        x = r_point['middle'][0]
        y = r_point['middle'][1]
        road.append((x, y))
    lines.append(LineString(road))

network = MultiLineString(lines)
display(network)


In [2]:
import pickle
from pathlib import Path
from os.path import join

working_dir = join(Path.home(), "Documents", 'BeamNG.drive')
fpath = Path(join(working_dir, 'west_coas_roads.bytes'))
# unpickle
with fpath.open('rb') as fhandle:
    road_spec = pickle.load(fhandle)
